In [1]:
# from time import sleep
# from random import randint
from bs4 import BeautifulSoup
# import datetime
import requests
import pandas as pd
import re

In [2]:
def get_page_source(url):
    agent = {"User-Agent":"Mozilla/5.0"} # User Agent defined to avoid robot detection
    source=requests.get(url, headers=agent)
    return source.text, source.status_code

def get_soup(content):
    soup=BeautifulSoup(content, "lxml")
    return soup

In [208]:
def get_links(url = "https://thebanks.eu/banks-by-country",
             base_url = "https://thebanks.eu"):
    text, status = get_page_source(url)
    soup = get_soup(text)
    div = soup.find('div', attrs = {'class' : 'col-xs-12'})
    div_all = div.findAll('div', attrs = {'class' : 'row table-row'})
    data = list()
    for div in div_all:
        a = div.find('a')
        if(a):
            link = a.get('href')
            link = base_url + link
            country = a.text.strip()
            bank = div.find('div', attrs = 
                             {'class' : 'table-col col-md-3 col-lg-3 col-sm-6 col-xs-6'})
            bank = bank.find('span').text.strip()
            operating_banks = int(re.findall('[0-9]+', bank)[0])
            data.append([link, country, operating_banks])
    return data

In [209]:
data = get_links()

In [231]:
def scrape_type_1(link, country):
    text, status = get_page_source(link)
    soup = get_soup(text)
    div = soup.find('div', attrs = {'class' : 'row', 'id' : 'operating_banks'})
    div_all = div.findAll('div', attrs = {'class' : 'rounded'})
    country_data = list()
    for div in div_all:
        bank_name = div.find('h4').text.strip()
        business_focus = div.find('p').text.strip()
#         total_assests = div.findAll('p')
        div = div.find('div',  attrs = {'class' : 'align-center'})
        country_data.append({'Country' : country,
                   'Bank Name' : bank_name,
                   'Business Focus' : business_focus}) 
    return country_data
    
    

In [232]:
def scrape_type_2(link, country):
    text, status = get_page_source(url)
    soup = get_soup(text)
    div = soup.find('div', attrs = {'class' : 'table-sort-body'})
    div_all = div.findAll('div', attrs = {'class' : 'row table-row table-sort-row'})
    country_data = list()
    for row in div_all:
        bank_name = row.findAll('a')[1].text.strip()
        business_focus = row.find('div', class_ = 'table-col business_focus col-sm-3 col-md-3 col-xs-6').text.strip()
        country_data.append({'Country' : country,
               'Bank Name' : bank_name,
               'Business Focus' : business_focus})
    return country_data


In [233]:
result = list()
for link, country, operating_banks in data:
    if(operating_banks < 30):
        result.extend(scrape_type_1(link, country))
    else:
        result.extend(scrape_type_2(link, country))
df = pd.DataFrame(result)

In [234]:
df.to_csv("European_banks.csv")
df

,Country,Bank Name,Business Focus
0,Andorra,"Andorra Banc Agrícol Reig, SA","International private banking, domestic retail..."
1,Andorra,"Banc Sabadell d’Andorra, SA","International private banking, domestic retail..."
2,Andorra,"Crèdit Andorrà, SA","International private banking, domestic retail..."
3,Andorra,"Mora Banc Grup, SA","International private banking, domestic retail..."
4,Andorra,"Vall Banc, SAU (new)","International private banking, domestic retail..."
...,...,...,...
3583,United Kingdom,Walser Privatbank AG,private banking and asset management
3584,United Kingdom,Western Union International Bank GmbH,business payment solutions
3585,United Kingdom,Wiener Privatbank SE,"private banking, asset management"
3586,United Kingdom,WSK Bank AG,retail banking


In [ ]:
# import re
# # url = "https://thebanks.eu/banks-by-country"
# # text, status = get_page_source(url)
# # soup = get_soup(text)
# # div = soup.find('div', attrs = {'class' : 'align-left container-fluid nice-resp-table'})
# # div_all = div.findAll('div', attrs = {'class' : 'row table-row'})
# data = list()
# for div in div_all:
#     a = div.find('a')
#     if(a):
#         link = a.get('href')
# #         link = base_url + link
#         country = a.text.strip()
#         banks = div.find('div', attrs = {'class' : 'table-col col-md-3 col-lg-3 col-sm-6 col-xs-6'})
# #         print(banks)
#         test = banks.find('span').text.strip()
# #         test = test.text.strip()
#         operating_banks = int(re.findall('[0-9]+', test)[0])
# #         print(test)
# #         test.find()
#         data.append([link, country, operating_banks])

text = 'Total Assets (2019): 44,574.51 mln EUR\xa0\xa0(+2.96%)'
text = re.sub(',', repl = '', string = text)
# s = re.findall("(\d+(\.\d+)?)", text)
text[text.find(':') : ]
# re.findall(":\s*xa", text)